In [1]:
from sklearn import datasets
from skimage.util import view_as_windows
from tensorflow.keras.datasets import cifar10 
from pixelhop2 import * 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from llsr import LLSR as myLLSR
from lag import * 
from tensorflow.keras.datasets import cifar10 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot 
from matplotlib import *
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from cross_entropy import *
from sklearn.ensemble import RandomForestClassifier
import time

In [2]:
# get data 
(train_set,train_label),(test_set,test_label) = cifar10.load_data() 
# class number 

print(train_label.shape)


(50000, 1)


In [3]:
def class_balance_data_set_split(data_set,label_set,nClasses,split_coef):
    # test balance data set 
    nClasses = 10 
    data_set_split = []
    label_set_split = []
    print(train_label==9)
    print(np.where(label_set==9)[0])
    for i in range(nClasses):
        data_set_class = data_set[np.where(label_set==i)[0],:,:,:]
        data_set_class_split,_,label_set_class_split, _ = train_test_split(data_set_class,label_set[train_label==i],train_size = split_coef)
        data_set_split.append(data_set_class_split)
        label_set_split.append(label_set_class_split)
    data_set_split = np.concatenate(data_set_split,axis = 0 )
    label_set_split = np.concatenate(label_set_split,axis = 0)
    print(data_set_split.shape)
    return data_set_split,label_set_split

In [4]:
# test balance data set 
nClasses = 10 
# train_data_split = []
# train_label_split = []
# print(train_label==9)
# print(np.where(train_label==9)[0])
# for i in range(nClasses):
#     train_data_class = train_set[np.where(train_label==i)[0],:,:,:]
#     train_data_class_split,_,train_label_class_split, _ = train_test_split(train_data_class,train_label[train_label==i],train_size = 0.2)
#     train_data_split.append(train_data_class_split)
#     train_label_split.append(train_label_class_split)
# train_data_split = np.concatenate(train_data_split,axis = 0 )
# train_label_split = np.concatenate(train_label_split,axis = 0)
# print(train_data_split.shape)



In [5]:
# shrink and concat function 
# example callback function for collecting patches and its inverse
def Shrink(X, shrinkArg):
    win = shrinkArg['filterwin']
    stride = shrinkArg['stride']
    poolwin = shrinkArg['poolwin']
    depth = shrinkArg['depth']
   
    # print("Shrink function")
    print("shape of original X:",X.shape)
    X = view_as_windows(X, (1,win,win,1), (1,stride,stride,1))
    # print("shape of windowed X: ",X.shape)
    X = X.reshape(X.shape[0], X.shape[1], X.shape[2], -1)
    print(X.shape)
    return X

def nonShrink(X, shrinkArg) : 
    return X
# example callback function for how to concate features from different hops
def Concat(output, concatArg):
    #print("the shape of final output is ", len(output))
#    for i in range(len(output)-1):
    #for i in range(len(output)):
     #   output[i].reshape(output[i].shape[0],-1)
    S = output[0].shape
    for i in range(len(output)):
        output[i] = output[i].reshape(S[0],-1)
    return output

In [6]:
# test shrink and concat function 
X = np.array([ [ [ [1, 2],[ 3, 4 ] ], [[ 5, 6  ],[ 7, 8 ]] ] , 
               [ [[ 9, 10 ],[ 11, 12  ]] , [[ 13, 14 ],[ 15 ,16 ] ]  ]  ])

# print(X.shape)
shrinkArgs = [{'func':Shrink,'poolwin':2,'method':np.max,'filterwin':5,'stride':1,'depth':1}, 
              {'func':Shrink,'poolwin':2,'method':np.mean,'filterwin':5,'stride':1,'depth':2},
              {'func':Shrink,'poolwin':2,'method':np.mean,'filterwin':4,'stride':1,'depth':3}] 
# Shrink(X,shrinkArgs[0])
concatArg = {'func':Concat}
# print(X.shape)
# print(X)
# print(train_set.shape)
# train_set_shrunken = Shrink(train_set,shrinkArgs[0])
# train_set_shrunken_2 = Shrink(train_set,shrinkArgs[1])
# con = [] 
# con.append(train_set_shrunken)
# con.append(train_set_shrunken_2)
# print(np.concatenate(con,axis=-1).shape)


In [7]:
SaabArgs = [{'num_AC_kernels':-1, 'needBias':False, 'useDC':True, 'batch':None, 'cw':False}, 
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'batch':None, 'cw':True, 'stride':1,'win':3},
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'batch':None, 'cw':True, 'stride':1,'win':3},
            {'num_AC_kernels':-1, 'needBias':True, 'useDC':True, 'batch':None, 'cw':True, 'stride':1,'win':3}]
shrinkArgs = [{'func':Shrink   , 'poolwin':2, 'stride':1 ,'filterwin':5,'depth':1,'method':np.max}, 
              {'func':Shrink   , 'poolwin':2, 'stride':1 ,'filterwin':5,'depth':2,'method':np.mean},
              {'func':Shrink   , 'poolwin':1, 'stride':1 ,'filterwin':3,'depth':3,'method':np.mean},]
concatArg = {'func':Concat,'depth':3}


In [8]:
pixelhop_unit = Pixelhop2(depth=3, TH1=0.001, TH2=0.0001, SaabArgs=SaabArgs, shrinkArgs=shrinkArgs, concatArg=concatArg)
modulus1_st = time.time()
pixelhop_unit.fit(train_set)
modulus1_et = time.time() 
modulus1_training_time = modulus1_et - modulus1_st 
print(modulus1_training_time)

pixelhop2 fit
shape of original X: (50000, 32, 32, 3)


C:\Users\12414\Anaconda3\lib\site-packages\skimage\util\shape.py:246: RuntimeWarning: Cannot provide views on a non-contiguous input array without copying.
  warn(RuntimeWarning("Cannot provide views on a non-contiguous input "


(50000, 28, 28, 75)
fit finish
batch : 0
batch : 1
batch : 2
batch : 3
batch : 4
current layer 0


KeyError: 'method'

In [ ]:

modulus2_st = time.time()
train_pixelhop_tran_set =  pixelhop_unit.transform(train_set)

# train_data_pixelhop = np.concatenate(output,axis= -1)
infer_time_m1_st = time.time()
test_pixelhop_tran_set = pixelhop_unit.transform(test_set)
infer_time_m1_ed = time.time()
infer_time_m1 = infer_time_m1_ed - infer_time_m1_st
# test_data_pixelhop = np.concatenate(output,axis =-1 )

In [ ]:
# feature selection based on cross entropy 
N_C = [0.6,0.7,0.8 ] 
num_cluster = 10
num_clusters_parm = [] 
infer_time_m2_cross = 0 
for layer in range(len(train_pixelhop_tran_set)):
    cross_entropy = Cross_Entropy(num_class=10, num_bin=num_cluster)
    train_pixelhop_tran = train_pixelhop_tran_set[layer]
    num_features = int(train_pixelhop_tran.shape[-1]*N_C[layer])
    feature_cross_entropy = cross_entropy.KMeans_Cross_Entropy(train_pixelhop_tran, train_label)
    feature_select = np.argsort(feature_cross_entropy)[:num_features]  
    train_pixelhop_tran_set[layer] = train_pixelhop_tran_set[layer][:,feature_select]
    infer_time_m2_cross_st = time.time()
    test_pixelhop_tran_set[layer] = test_pixelhop_tran_set[layer][:,feature_select]
    infer_time_m2_cross_ed = time.time()
    infer_time_m2_cross+=infer_time_m2_cross_ed-infer_time_m2_cross_st


In [ ]:
infer_time_m2_lag = 0 
for i in range(10):
    num_clusters_parm.append(num_cluster)
train_lag_tran_set = [] 
test_lag_tran_set = []
for layer in range(len(train_pixelhop_tran_set)):
    lag = LAG(encode='distance', num_clusters= num_clusters_parm, alpha=7, learner=myLLSR(onehot=False))  
    print(train_pixelhop_tran_set[layer].shape)
    lag.fit(train_pixelhop_tran_set[layer],train_label)
    train_lag_tran = lag.transform(train_pixelhop_tran_set[layer])
    train_lag_tran_set.append(train_lag_tran)
    infer_time_m2_lag_st = time.time()
    test_lag_tran = lag.transform(test_pixelhop_tran_set[layer])
    test_lag_tran_set.append(test_lag_tran)
    infer_time_m2_lag_ed = time.time()
    infer_time_m2_lag +=  infer_time_m2_lag_ed - infer_time_m2_lag_st

In [ ]:
# feature selection based on cross entropy 
# N_C = 0.5

# for layer in range(len(train_pixelhop_tran_set)):
 #  cross_entropy = Cross_Entropy(num_class=10, num_bin=num_cluster)
 #  train_lag_tran = train_lag_tran_set[layer]
 #  feature_cross_entropy = np.zeros(train_lag_tran.shape[-1])
 #  num_features = int(train_lag_tran.shape[-1]*N_C)
 #  for k in range(train_lag_tran.shape[-1]):
 #      feature_cross_entropy[k] = cross_entropy.KMeans_Cross_Entropy(train_lag_tran[:,k].reshape(-1,1), train_label)
 #  feature_select = np.argsort(feature_cross_entropy)[:num_features]  
 #  train_lag_tran_set[layer] = train_lag_tran_set[layer][:,feature_select]
 #  test_lag_tran_set[layer] = test_lag_tran_set[layer][:,feature_select]

In [ ]:
print(train_lag_tran_set[0].shape)
train_tran_classifier = np.concatenate(train_lag_tran_set,-1)
test_tran_classifier = np.concatenate(test_lag_tran_set,-1)
modulus2_et = time.time()
modulus2_training_time = modulus2_et - modulus2_st 
print("modulus2 training time", modulus2_training_time)


In [ ]:
#svm_classifier = SVC(C =1,gamma = 10, kernel='rbf' )
#svm_classifier.fit(train_tran_classifier,train_label)
modulus3_st = time.time()
random_forest = RandomForestClassifier(n_estimators=600, criterion='gini')
random_forest.fit(train_tran_classifier,train_label)

In [ ]:
train_pred = random_forest.predict(train_tran_classifier)
infer_time_m3_classifier = 0 
infer_time_m3_classifier_st = time.time()
test_pred = random_forest.predict(test_tran_classifier)
infer_time_m3_classifier_ed = time.time()
infer_time_m3_classifier = infer_time_m3_classifier_ed - infer_time_m3_classifier_st

In [ ]:
print(accuracy_score(train_label,train_pred))
print(accuracy_score(test_label,test_pred))

In [ ]:
total_infer_time_test = infer_time_m1 + infer_time_m2_cross + infer_time_m2_lag + infer_time_m3_classifier
print("total inference time for test data set", total_infer_time_test)
train_confusion_matrix = confusion_matrix(train_label, train_pred)
test_confusion_matrix = confusion_matrix(test_label,test_pred)
modulus3_et = time.time() 
modulus3_training_time = modulus3_et - modulus3_st 
print("modulus3 training time", modulus3_training_time)
pyplot.figure()
pyplot.imshow(train_confusion_matrix)
pyplot.figure()
pyplot.imshow(test_confusion_matrix)
print("------- DONE -------\n")
print(train_confusion_matrix)
print(test_confusion_matrix)